In [1]:
#!/usr/bin/env python
# coding: utf-8
'''
To get information in each board.

getArticlesInfo(board_url: str,
             start_date: str('4/10'),
             end_date: str('4/15')) -> List( Dict[article_title, article_url, auth_id, date, article_content, comment_content] )
    Return a list of dictionaries which contain keys of article_url, auth_id and date.

getContent(article_url: str) -> Dict[article_title, article_url, auth_id, auth_name, date, article_content, comment_content]

listComments(comment_content: str) -> Dict[comment_id]: List( Dict[push_tag: str, comment: str, date: time] )

convertArticlesInfo( all_articles: List[ Dict[] ] ) -> List[ Tuple[str] ]
    The input shoud be like the data type which is returned by getArticlesInfo()
    Return a list of tuples, which can be insert into PTT_ARTICLE

convertCommentsContent( comment_dict: List[ Dict[ push-tag, push-userid, push-content, push-ipdatetime ] ] )
    
isArticleIdInDb( article_id: str ) -> Bool

'''

from kafka import KafkaProducer, KafkaClient
import requests
from bs4 import BeautifulSoup
import random
import time
import datetime
import logging
import json

import connectDB

datetime_now = datetime.datetime.now()
datetime_1970 = datetime.datetime(1970, 1, 1)
end_date = datetime_now.strftime('%Y/%m/%d')
start_date = datetime_1970.strftime('%Y/%m/%d')

exist_article_id = connectDB.getAllArticleId()

# Set headers
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36'}
cookies = {'over18': '1'}

# The format of date should be like 2020/04/15
def getArticlesInfo(board_url, start_date=start_date, end_date=end_date):
    all_articles = list()
    all_articles_index = 0
    one_page_articles = list()
    board_name = board_url.split('/')[-2]
    # Each id will be like Gossiping_M.1587040493.A.58B.html
    exist_board_article_id = connectDB.getAllArticleIdInOneBoard(board_name)
    
    # Insert the status into PTT_ETL_LOG
    process_id = board_name + str(int(time.time()))
    etl_dt = datetime_now.strftime("%Y-%m-%d %H:%M:%S")
    record_dt = datetime_now.strftime("%Y-%m-%d %H:%M:%S")
    crawled_range = """({},{})""".format(start_date, end_date)
    etl_status = 'start'
    connectDB.insertIntoPttEtlLog(process_id, etl_dt, record_dt, crawled_range, etl_status)
    
    start_date = datetime.datetime.strptime(start_date, '%Y/%m/%d')
    end_date = datetime.datetime.strptime(end_date, '%Y/%m/%d') + datetime.timedelta(days=1)
    print('start_date:', start_date)
    print('end_date:', end_date)
    print('===============')
    ss = requests.session()
    ss.cookies['over18'] = '1'
    res = ss.get(board_url, headers=headers)
    soup = BeautifulSoup(res.text, 'html.parser')
    
    last_page_url = 'https://www.ptt.cc'
    article_created_date = start_date
    
    # Insert the status into PTT_ETL_LOG
    #process_id = board_name + str(int(time.time()))
    #etl_dt = datetime_now.strftime("%Y-%m-%d %H:%M:%S")
    record_dt = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    #crawled_range = """({},{})""".format(start_date, end_date)
    etl_status = 'executing'
    connectDB.insertIntoPttEtlLog(process_id, etl_dt, record_dt, crawled_range, etl_status)
    # Produce detail log to Kafka and DB
    etl_detail_status = 'INFO'
    etlStatusMessage = f'Crawler target: {board_url}'
    produceLogToKafkaAndDb(process_id=process_id, etl_status=etl_detail_status, etl_status_message=etlStatusMessage)
  
    # The date of which article was created should be greater than or equal to start_date
    while last_page_url != '' and article_created_date >= start_date:
        too_early_article_count = 0 # To count the article amount whose created date is earlier than start_date
        
        # index1.html does not have 'href' key
        try:
            last_page_url = 'https://www.ptt.cc' + \
                            soup.select('div[class="btn-group btn-group-paging"] a.btn')[1]['href']   
            # Produce detail log to Kafka and DB
            etl_detail_status = 'INFO'
            etlStatusMessage = f'Get last page url: {last_page_url}'
            produceLogToKafkaAndDb(process_id=process_id, etl_status=etl_detail_status, etl_status_message=etlStatusMessage)
        except KeyError:
            last_page_url = ''
            article_created_date = start_date
            # Produce detail log to Kafka and DB
            etl_detail_status = 'WARN'
            etlStatusMessage = f'No last page.'
            produceLogToKafkaAndDb(process_id=process_id, etl_status=etl_detail_status, etl_status_message=etlStatusMessage)
        titles_soup_list = soup.select('div.r-ent')
        for title_soup in titles_soup_list:
            try:
                article_url = 'https://www.ptt.cc' + title_soup.select('div.title a')[0]['href']
                # Produce detail log to Kafka and DB
                etl_detail_status = 'INFO'
                etlStatusMessage = f'Get article url: {article_url}'
                produceLogToKafkaAndDb(process_id=process_id, etl_status=etl_detail_status, etl_status_message=etlStatusMessage)
            except IndexError as e:
                print(e.args)
                print(title_soup)
                # Produce detail log to Kafka and DB
                etl_detail_status = 'WARN'
                etlStatusMessage = '{}: {}'.format(e.args, title_soup)
                produceLogToKafkaAndDb(process_id=process_id, etl_status=etl_detail_status, etl_status_message=etlStatusMessage)
                continue
            article_title = title_soup.select('div.title a')[0].text
            print(article_title)
            print(article_url)
            
            # If article id exists, pass
            if '_'.join(article_url.split('/')[-2:]) in exist_board_article_id:
                print('Exists!')
                # Produce detail log to Kafka and DB
                etl_detail_status = 'INFO'
                etlStatusMessage = f'Article already exists: {article_title}[{article_url}]'
                produceLogToKafkaAndDb(process_id=process_id, etl_status=etl_detail_status, etl_status_message=etlStatusMessage)
            else:
                article_content = getContent(article_url, process_id)
                # If the article does not have 'date' infotmation, 
                # refer the date in last article to set for this article
                try:
                    article_created_date = article_content['date']
                    # Produce detail log to Kafka and DB
                    etl_detail_status = 'INFO'
                    etlStatusMessage = f'Try to get article created datetime: {article_title}[{article_created_date}]'
                    produceLogToKafkaAndDb(process_id=process_id, etl_status=etl_detail_status, etl_status_message=etlStatusMessage)
                except KeyError:
                    try:
                        article_content['date'] = all_articles[all_articles_index - 1]['date']
                        article_created_date = article_content['date']
                        # Produce detail log to Kafka and DB
                        etl_detail_status = 'WARN'
                        etlStatusMessage = f'Article datetime not exist: {article_title}, set it to be as the last article {article_created_date}'
                        produceLogToKafkaAndDb(process_id=process_id, etl_status=etl_detail_status, etl_status_message=etlStatusMessage)
                    # If there is no date information in last article, set it to be end_date
                    except:
                        article_content['date'] = end_date
                        article_created_date = article_content['date']
                        # Produce detail log to Kafka and DB
                        etl_detail_status = 'WARN'
                        etlStatusMessage = f'Article datetime not exist: {article_title}, set it to be end_date {end_date}'
                        produceLogToKafkaAndDb(process_id=process_id, etl_status=etl_detail_status, etl_status_message=etlStatusMessage)
                if article_created_date < start_date:
                    print('article_created_date < start_date')
                    too_early_article_count += 1
                    # Produce detail log to Kafka and DB
                    etl_detail_status = 'INFO'
                    etlStatusMessage = f'Article created date earlier than start_date: {article_title}[{article_created_date}]'
                    produceLogToKafkaAndDb(process_id=process_id, etl_status=etl_detail_status, etl_status_message=etlStatusMessage)
                    # Because there are some bottom articles that are not ordered by date, 
                    # maybe announcement or any other like that.
                    # The process will stop if it find out the first earlier-than-start-date-article
                    # without this condition, so if there are not so many such article, 
                    # the process is no need to stop.
                    if too_early_article_count > 20:
                        print('too_early_article_count > 20')
                        last_page_url = ''
                        # Produce detail log to Kafka and DB
                        etl_detail_status = 'WARN'
                        etlStatusMessage = 'No more matched articles'
                        produceLogToKafkaAndDb(process_id=process_id, etl_status=etl_detail_status, etl_status_message=etlStatusMessage)
                        break
                elif article_created_date > end_date:
                    print('article_created_date > end_date')
                    # Produce detail log to Kafka and DB
                    etl_detail_status = 'INFO'
                    etlStatusMessage = f'Article created date later than end_date: {article_title}[{article_created_date}]'
                    produceLogToKafkaAndDb(process_id=process_id, etl_status=etl_detail_status, etl_status_message=etlStatusMessage)
                    pass
                else:
                    all_articles.append(article_content)
                    one_page_articles.append(article_content)
                    print('Appended!')
                    # Produce detail log to Kafka and DB
                    etl_detail_status = 'INFO'
                    etlStatusMessage = f'Article matched: {article_title}[{article_created_date}]'
                    produceLogToKafkaAndDb(process_id=process_id, etl_status=etl_detail_status, etl_status_message=etlStatusMessage)
                time.sleep(random.randint(1, 100) / 50)
        
        ## Insert one page data into the two tables, PTT_ARTICLE and PTT_COMMENT
        insert_data = convertArticlesInfo(one_page_articles)
        connectDB.insertIntoPttArticle(insert_data)
        # Produce detail log to Kafka and DB
        etl_detail_status = 'INFO'
        etlStatusMessage = f'Articles content insert into PttArticle: {article_title}[{article_created_date}]'
        produceLogToKafkaAndDb(process_id=process_id, etl_status=etl_detail_status, etl_status_message=etlStatusMessage)
        insert_data = convertCommentsContent(one_page_articles)
        connectDB.insertIntoPttComment(insert_data)
        # Produce detail log to Kafka and DB
        etl_detail_status = 'INFO'
        etlStatusMessage = f'Articles comment content insert into PttComment: {article_title}[{article_created_date}]'
        produceLogToKafkaAndDb(process_id=process_id, etl_status=etl_detail_status, etl_status_message=etlStatusMessage)
        one_page_articles = list()
                
        print('last_page_url:', last_page_url)
        if last_page_url == '': 
            # Produce detail log to Kafka and DB
            etl_detail_status = 'INFO'
            etlStatusMessage = 'Completed: no more pages'
            produceLogToKafkaAndDb(process_id=process_id, etl_status=etl_detail_status, etl_status_message=etlStatusMessage)
            break
        
        # Get into the last page
        res = ss.get(last_page_url, headers=headers)
        soup = BeautifulSoup(res.text, 'html.parser')
        
        # Produce detail log to Kafka and DB
        etl_detail_status = 'INFO'
        etlStatusMessage = f'Crawler target: {last_page_url}'
        produceLogToKafkaAndDb(process_id=process_id, etl_status=etl_detail_status, etl_status_message=etlStatusMessage)
    
    # Insert the status into PTT_ETL_LOG
    #process_id = board_name + int(time.time())
    #etl_dt = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    record_dt = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    #crawled_range = """({},{})""".format(start_date, end_date)
    etl_status = 'end'
    connectDB.insertIntoPttEtlLog(process_id, etl_dt, record_dt, crawled_range, etl_status)
    # Produce detail log to Kafka and DB
    etl_detail_status = 'INFO'
    etlStatusMessage = 'Crawler successfully executed!'
    produceLogToKafkaAndDb(process_id=process_id, etl_status=etl_detail_status, etl_status_message=etlStatusMessage)

    return all_articles

def getContent(article_url, process_id):
    article_content = dict()
    
    article_content['article_url'] = article_url
    ss = requests.session()
    ss.cookies['over18'] = '1'
    reconnection_time = 0
    while True:
        try:
            res = ss.get(article_url, headers=headers)
            # Produce detail log to Kafka and DB
            etl_detail_status = 'INFO'
            etlStatusMessage = f'Crawler target: {article_url}'
            produceLogToKafkaAndDb(process_id=process_id, etl_status=etl_detail_status, etl_status_message=etlStatusMessage)
            break
        except:
            print('Connection refused by server!')
            print('Reconnecting!')
            # Produce detail log to Kafka and DB
            etl_detail_status = 'WARN'
            etlStatusMessage = f'Reconnect {reconnection_time+1} time(s): {article_url}'
            produceLogToKafkaAndDb(process_id=process_id, etl_status=etl_detail_status, etl_status_message=etlStatusMessage)
            time.sleep(random.randint(10, 20))
            reconnection_time += 1
            continue
    soup = BeautifulSoup(res.text, 'html.parser')

    # Get author information : 作者 -> 標題 -> 時間 -> 版名
    # Dict[article_url, auth_id, auth_name, date, article_content, comment_content]
    # These datas may not exist, so it is necessary to do try-catch
    try:
        article_content['auth_id'] = soup.select('div[class="article-metaline"]')[0] \
                                        .span.find_next_sibling() \
                                        .text \
                                        .split(' ')[0]
        article_content['auth_name'] = soup.select('div[class="article-metaline"]')[0] \
                                        .extract() \
                                        .span.find_next_sibling() \
                                        .text \
                                        .split(' ')[1][1:-1]
        article_content['title'] = soup.select('div[class="article-metaline"]')[0] \
                                        .extract() \
                                        .span.find_next_sibling() \
                                        .text 
        article_content['date'] = soup.select('div[class="article-metaline"]')[0] \
                                        .extract() \
                                        .span.find_next_sibling() \
                                        .text
        article_content['date'] = datetime.datetime.strptime(' '.join(article_content['date'].split(' ')[1:])
                                                             , '%b %d %X %Y')
        article_content['board_name'] = soup.select('div[class="article-metaline-right"]')[0] \
                                            .extract() \
                                            .span.find_next_sibling() \
                                            .text
    except IndexError:
        for k in ['auth_id', 'auth_name', 'title']:
            article_content[k] = 'Unknown'
        article_content['board_name'] = article_url.split('/')[-2]

    # Get article content
    article_content['article_content'] = soup.select('#main-content')[0].text.split('※ 發信站')[0]

    # Get the whole comments without classifying by comment ID
    article_content['comment_content'] = list()
    tmp_dict = dict()
    for tag in soup.select('#main-content')[0].select('span')[2:]:
        for comment_key in ['push-tag', 'push-userid', 'push-content', 'push-ipdatetime']:
            if comment_key in tag.attrs['class']:
                tmp_dict[comment_key] = tag.text.strip('\n')
                # The order of class in span-tag is ['push-tag', 'push-userid', 'push-content', 'push-ipdatetime']
                if comment_key == 'push-ipdatetime': # Reset tmp_dict()
                    article_content['comment_content'].append(tmp_dict)
                    tmp_dict = dict()
    # Produce detail log to Kafka and DB
    etl_detail_status = 'INFO'
    etlStatusMessage = f'Got article content: {article_url}'
    produceLogToKafkaAndDb(process_id=process_id, etl_status=etl_detail_status, etl_status_message=etlStatusMessage)
    
    return article_content

def listComments(comment_content):
    pass

# Convert the data returned from getArticlesInfo() to the format
# which can be successfully insert into PTT_ARTICLE
def convertArticlesInfo(all_articles):
    # This data can be the input of connectDB.insertIntoPttArticle(insert_data)
    insertData = list()
    for row in all_articles:
        try:
            up_tag = len([c['push-tag'] for c in row['comment_content'] if '推' in c['push-tag']])
            down_tag = len([c['push-tag'] for c in row['comment_content'] if '噓' in c['push-tag']])
        except:
            up_tag = 0
            down_tag = 0
        oneRowData = (
            row['title'],
            row['board_name'],
            '_'.join(row['article_url'].split('/')[-2:]),
            row['auth_id'],
            row['auth_name'],
            row['date'].strftime("%Y-%m-%d %H:%M:%S"),
            row['article_url'],
            row['date'].strftime("%Y-%m-%d %H:%M:%S"),
            row['date'].strftime("%Y-%m-%d %H:%M:%S"),
            up_tag,
            down_tag,
            row['article_content'],
            str(row['comment_content'])
        )
        insertData.append(oneRowData)
    return insertData

# Convert the data returned from getArticlesInfo()[i]['comment_content'] to the format
# which can be successfully insert into PTT_COMMENT
def convertCommentsContent(all_articles):
    # This data can be the input of connectDB.insertIntoPttComment(insert_data)
    insertData = list()
    for row in all_articles:
        try:
            article_id = '_'.join(row['article_url'].split('/')[-2:])
            for comment in row['comment_content']:
                comment_id = comment['push-userid']
                push_tag = comment['push-tag']
                comment_content = comment['push-content']
                comment_datetime = comment['push-ipdatetime']
                oneRowData = (
                    article_id,
                    comment_id,
                    push_tag,
                    comment_content,
                    comment_datetime
                )
                insertData.append(oneRowData)
        except:
            pass
    return insertData

def produceLogToKafkaAndDb(process_id, etl_status, etl_status_message):
    '''
    key: process_id
    value: etl_status|log_record_dt|etlStatusMessage
           WARNING|2020-04-20 21:19:53|Hello world
    '''
    # Load the configuration of Kafka
    with open('./kafka_producer.conf', 'r', encoding='utf-8') as f:
        kafkaconf = json.loads(f.read())
        
    # New a Kafka producer instance, then set up the configuration for Kafka connection
    producer = KafkaProducer(
        # Kafka cluster
        bootstrap_servers=kafkaconf['bootstrap_servers'],
        key_serializer=str.encode,
        value_serializer=str.encode
    )
    
    log_record_dt = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # Topic name
    topic_name = kafkaconf['topic']
    key = process_id
    value = f'{etl_status}|{log_record_dt}|{etl_status_message}'
    
    # Produce log to Kafka
    try:
        producer.send(topic=topic_name, key=key, value=value)
        print('KAFKA PRODUCED')
    except:
        print('KAFKA PASS')
        pass
    finally:
        producer.close()
    
    # Insert log into DB
    connectDB.insertIntoPttEtlDetailLog(process_id, log_record_dt, etl_status, etl_status_message)
    
    return 1

def isArticleIdInDb(article_id):
    pass

def isBoardArticleIdInDb(article_id, board_name):
    pass

if __name__ == '__main__':
    board_url = 'https://www.ptt.cc/bbs/Gossiping/index.html'
    test_data = getArticlesInfo(board_url, start_date='2020/04/19', end_date='2020/04/21')
    insert_data = convertArticlesInfo(test_data)
    print(len(test_data))
    print(len(insert_data))
#     connectDB.insertIntoPttArticle(insert_data)
    insert_data = convertCommentsContent(test_data)
    print(len(insert_data))
#     connectDB.insertIntoPttComment(insert_data)

Successfully connected!
Successfully connected!
Successfully connected!
start_date: 2020-04-19 00:00:00
end_date: 2020-04-22 00:00:00
Successfully connected!
KAFKA PRODUCED
Successfully connected!
KAFKA PRODUCED
Successfully connected!
KAFKA PRODUCED
Successfully connected!
[問卦] 只有我被八卦版的迷信嚇到了嗎？
https://www.ptt.cc/bbs/Gossiping/M.1587512509.A.A6B.html
KAFKA PRODUCED
Successfully connected!
KAFKA PRODUCED
Successfully connected!
KAFKA PRODUCED
Successfully connected!
article_created_date > end_date
KAFKA PRODUCED
Successfully connected!
KAFKA PRODUCED
Successfully connected!
[新聞] 川普：暫停發放綠卡60天 
https://www.ptt.cc/bbs/Gossiping/M.1587512515.A.676.html
KAFKA PRODUCED
Successfully connected!
KAFKA PRODUCED
Successfully connected!
KAFKA PRODUCED
Successfully connected!
article_created_date > end_date
KAFKA PRODUCED
Successfully connected!
KAFKA PRODUCED
Successfully connected!
[問卦] 驚!!!藥局前面沒人在排隊買口罩
https://www.ptt.cc/bbs/Gossiping/M.1587512633.A.F09.html
KAFKA PRODUCED
Successfully connected!

KeyboardInterrupt: 

In [2]:
board_url = 'https://www.ptt.cc/bbs/Gossiping/index.html'
test_data = getArticlesInfo(board_url, start_date='2020/4/16')

start_date: 2020-04-16 00:00:00
end_date: 2020-04-18 00:00:00
https://www.ptt.cc
Re: [新聞] 國台辦：證據來自武漢衛健委 台先示警是
https://www.ptt.cc/bbs/Gossiping/M.1587139202.A.1BE.html
Appended!
[問卦] 渡邊理佐與6/6登陸之日
https://www.ptt.cc/bbs/Gossiping/M.1587139207.A.126.html
Appended!
[問卦] 白橋、橋花、橋鳥、橋麥...誰才是真愛?
https://www.ptt.cc/bbs/Gossiping/M.1587139214.A.397.html
article_created_date > end_date
[問卦] 未央奈的自拍
https://www.ptt.cc/bbs/Gossiping/M.1587139219.A.1C3.html
article_created_date > end_date
[問卦] 國中會尻槍的比例多高
https://www.ptt.cc/bbs/Gossiping/M.1587139228.A.126.html
article_created_date > end_date
[問卦] 愛は感情の忘れもの
https://www.ptt.cc/bbs/Gossiping/M.1587139242.A.09B.html
article_created_date > end_date
Re: [爆卦] 香港傳染疾病控制專家到武漢一日後逃離
https://www.ptt.cc/bbs/Gossiping/M.1587139244.A.CE7.html
article_created_date > end_date
[問卦] 偶像怎麼知道來握手的粉絲昰不是主推她?
https://www.ptt.cc/bbs/Gossiping/M.1587139277.A.786.html
article_created_date > end_date
[問卦] 加藤史帆要我一起參加TGC?~
https://www.ptt.cc/bbs/Gossiping/M.1587139278.A.5DF.html
arti

Appended!
[公告] 盒盒水桶囉
https://www.ptt.cc/bbs/Gossiping/M.1587137101.A.A57.html
Appended!
[問卦] 大家持修算正嗎？
https://www.ptt.cc/bbs/Gossiping/M.1587137107.A.828.html
Appended!
[問卦] 館長是不是道歉大隊長？
https://www.ptt.cc/bbs/Gossiping/M.1587137131.A.305.html
Appended!
[問卦] 為什麼內褲會結塊
https://www.ptt.cc/bbs/Gossiping/M.1587137157.A.D62.html
Appended!
Re: [問卦] 有沒有遇過好的中國人
https://www.ptt.cc/bbs/Gossiping/M.1587137215.A.F72.html
Appended!
Re: [問卦] 馬華跟新加坡華為什麼那麼愛支那
https://www.ptt.cc/bbs/Gossiping/M.1587137274.A.1C3.html
Appended!
[問卦] 有沒有遇過好的中國人
https://www.ptt.cc/bbs/Gossiping/M.1587137305.A.FE0.html
Appended!
[政治] 王浩宇自揭賣襪子辛酸：我一個人在店裡哭
https://www.ptt.cc/bbs/Gossiping/M.1587137307.A.FB2.html
Appended!
[問卦] 躲在廁所靜靜是不是很爽？
https://www.ptt.cc/bbs/Gossiping/M.1587137311.A.F75.html
Appended!
Re: [新聞] 欠稅11萬 台南2500萬金店面遭法拍
https://www.ptt.cc/bbs/Gossiping/M.1587137315.A.F8D.html
Appended!
Re: [新聞] Gogoro非商用要車主舉證 消基會：不合理
https://www.ptt.cc/bbs/Gossiping/M.1587137322.A.B5B.html
Appended!
[問卦] 什麼時候買東西比較便宜？
https://www.pt

Appended!
[問卦] 喝酒肛門會癢
https://www.ptt.cc/bbs/Gossiping/M.1587135078.A.2CE.html
Appended!
[問卦] 你看啊春日的蝴蝶 你看它顫抖著飛越?
https://www.ptt.cc/bbs/Gossiping/M.1587135091.A.674.html
Appended!
[問卦]  育碧有中資入股嗎????
https://www.ptt.cc/bbs/Gossiping/M.1587135108.A.07D.html
Appended!
[問卦] 吾輩是貓(磨蹭)
https://www.ptt.cc/bbs/Gossiping/M.1587135116.A.4B3.html
Appended!
Re: [問卦] 為甚麼批評支那政府如殺支那人父母
https://www.ptt.cc/bbs/Gossiping/M.1587135143.A.C53.html
Appended!
[政治] 要求學生註冊「抖音」 賴品妤：新北市教
https://www.ptt.cc/bbs/Gossiping/M.1587135147.A.85D.html
Appended!
[問卦] 理組的圖書館地縛靈是哪些科系?
https://www.ptt.cc/bbs/Gossiping/M.1587135233.A.6C6.html
Appended!
[問卦] RMB還是CNY?
https://www.ptt.cc/bbs/Gossiping/M.1587135240.A.3A6.html
Appended!
[問卦] RO仙境傳說主題曲當年有多紅？
https://www.ptt.cc/bbs/Gossiping/M.1587135280.A.8EA.html
Appended!
Re: [新聞] 全台最美高中生出爐！景美正妹甜笑：能當
https://www.ptt.cc/bbs/Gossiping/M.1587135281.A.759.html
Appended!
Re: [新聞] 欠罰單1萬8房被法拍賤賣老母到官署下跪
https://www.ptt.cc/bbs/Gossiping/M.1587134160.A.A39.html
Appended!
[新聞] 拿動物餵動物 德國動物園擬

Appended!
[爆卦] 5ch評"韓國在野粉選後崩潰喊移民"
https://www.ptt.cc/bbs/Gossiping/M.1587133070.A.4B6.html
Appended!
[新聞] G7領袖視訊會議 共抗武漢肺炎疫情並呼籲
https://www.ptt.cc/bbs/Gossiping/M.1587131748.A.834.html
Appended!
Re: [問卦] 石油太便宜  台灣店家準備調降價格 真的嗎
https://www.ptt.cc/bbs/Gossiping/M.1587131754.A.E3B.html
Appended!
[問卦] 長的太好看整形醫生不收？
https://www.ptt.cc/bbs/Gossiping/M.1587131758.A.FAB.html
Appended!
[政治] 小英邀世界看中職 特地回應猿粉美主播
https://www.ptt.cc/bbs/Gossiping/M.1587131759.A.1FB.html
Appended!
[問卦] 肥宅484該正名為A5男孩?
https://www.ptt.cc/bbs/Gossiping/M.1587131769.A.6FE.html
Appended!
[問卦] 51勞動節計畫去哪裡玩？
https://www.ptt.cc/bbs/Gossiping/M.1587131839.A.F15.html
Appended!
[問卦] 慟!!!越南正妹店員對我視若無睹怎麼辦?
https://www.ptt.cc/bbs/Gossiping/M.1587131847.A.5BA.html
Appended!
[問卦] 認真問 誰想買AAPL的SE2?
https://www.ptt.cc/bbs/Gossiping/M.1587131856.A.759.html
Appended!
Re: [政治] 「華航改名 快！」黃智賢酸：改「台灣」
https://www.ptt.cc/bbs/Gossiping/M.1587131905.A.E67.html
Appended!
[問卦] 慟！現任玉皇關二爺駕崩！
https://www.ptt.cc/bbs/Gossiping/M.1587131923.A.C15.html
Appende

[問卦] 這是什麼蜘蛛
https://www.ptt.cc/bbs/Gossiping/M.1587129349.A.D52.html
Appended!
[問卦] 新訓感冒可能產生武漢病毒抗體嗎？
https://www.ptt.cc/bbs/Gossiping/M.1587129352.A.565.html
Appended!
Re: [問卦] 口罩不用排隊了 販賣機該功成身退了吧
https://www.ptt.cc/bbs/Gossiping/M.1587129356.A.801.html
Appended!
[問卦] 4/23開始買口罩不用再排隊了？
https://www.ptt.cc/bbs/Gossiping/M.1587129368.A.380.html
Appended!
Re: [政治] 武漢副市長：希望大家來玩
https://www.ptt.cc/bbs/Gossiping/M.1587129374.A.21C.html
Appended!
Re: [問卦] 大學畢業典禮在6/6怎麼辦
https://www.ptt.cc/bbs/Gossiping/M.1587129405.A.E3F.html
Appended!
[問卦] Ptt台男平均拿到多少遺產？
https://www.ptt.cc/bbs/Gossiping/M.1587129431.A.223.html
Appended!
[新聞] 富二代學生開瑪莎拉蒂放學撞翻小貨車 輔大前釀數人傷
https://www.ptt.cc/bbs/Gossiping/M.1587129473.A.CCD.html
Appended!
[問卦] 這是甚麼原理？
https://www.ptt.cc/bbs/Gossiping/M.1587129475.A.C42.html
Appended!
[問卦] 在臺灣開民用悍馬都哪種人？
https://www.ptt.cc/bbs/Gossiping/M.1587129509.A.D2F.html
Appended!
[問卦] 有沒有被盜墓的掛?
https://www.ptt.cc/bbs/Gossiping/M.1587129540.A.CE7.html
Appended!
[問卦] 韓國人會想統一嗎？
https://www.ptt.cc/bb

...

Appended!
('list index out of range',)
<div class="r-ent">
<div class="nrec"></div>
<div class="title">
			
				(本文已被刪除) [afflic]
			
			</div>
<div class="meta">
<div class="author">-</div>
<div class="article-menu">
</div>
<div class="date"> 4/16</div>
<div class="mark"></div>
</div>
</div>
('list index out of range',)
<div class="r-ent">
<div class="nrec"><span class="hl f3">20</span></div>
<div class="title">
			
				(本文已被刪除) [yokann]
			
			</div>
<div class="meta">
<div class="author">-</div>
<div class="article-menu">
</div>
<div class="date"> 4/16</div>
<div class="mark"></div>
</div>
</div>
[問卦] 欸欸 很累怎麼辦
https://www.ptt.cc/bbs/Gossiping/M.1586993339.A.A80.html
Appended!
[問卦] 開火車跟高鐵應該很無聊吧？
https://www.ptt.cc/bbs/Gossiping/M.1586993668.A.3FF.html
Appended!
Re: [問卦] 高雄同條路一鋪再鋪的掛
https://www.ptt.cc/bbs/Gossiping/M.1586993678.A.6CE.html
Appended!
Re: [問卦] 中國真能打下台灣？
https://www.ptt.cc/bbs/Gossiping/M.1586993699.A.80D.html
Appended!
Re: [問卦] 中國真能打下台灣？
https://www.ptt.cc/bbs/Gossiping/

[問卦] 不然你們這兩天又看了什麼世界名著??
https://www.ptt.cc/bbs/Gossiping/M.1586979575.A.7F2.html
Appended!
Re: [政治] 中職／蘇貞昌呼籲改名 會長吳志揚：我們
https://www.ptt.cc/bbs/Gossiping/M.1586979780.A.F0B.html
Appended!
[爆卦] 香港第一youtuber評論阿滴廣告
https://www.ptt.cc/bbs/Gossiping/M.1586980065.A.E96.html
Appended!
[問卦] 友梅友等待的時空有點重的卦
https://www.ptt.cc/bbs/Gossiping/M.1586980236.A.EEA.html
Appended!
Re: [問卦] 新加坡為何最近疫情開始大爆發？
https://www.ptt.cc/bbs/Gossiping/M.1586980474.A.12F.html
Appended!
[問卦] 自己住的人養寵物不會覺得牠們很可憐嗎？
https://www.ptt.cc/bbs/Gossiping/M.1586980522.A.64D.html
Appended!
Re: [問卦] 中國真能打下台灣？
https://www.ptt.cc/bbs/Gossiping/M.1586980733.A.6C6.html
Appended!
Re: [問卦] 中國真能打下台灣？
https://www.ptt.cc/bbs/Gossiping/M.1586981409.A.657.html
Appended!
[問卦] 石油危機
https://www.ptt.cc/bbs/Gossiping/M.1586981633.A.483.html
Appended!
Re: [問卦] 中國真能打下台灣？
https://www.ptt.cc/bbs/Gossiping/M.1586982063.A.FAC.html
Appended!
Re: [問卦] 中國真能打下台灣？
https://www.ptt.cc/bbs/Gossiping/M.1586982129.A.307.html
Appended!
[問卦] 泰式絲襪珍珠奶茶會好喝嗎
https://www.p

Appended!
[問卦] 我的生命這幾天變得非常的奇怪…
https://www.ptt.cc/bbs/Gossiping/M.1586973684.A.5C4.html
Appended!
[新聞] 釋昭慧法師不安！「武漢人不喜歡就不能
https://www.ptt.cc/bbs/Gossiping/M.1586973722.A.7C0.html
Appended!
[問卦] 自我矮化來凸顯台灣有比較不好嗎？
https://www.ptt.cc/bbs/Gossiping/M.1586973770.A.40E.html
Appended!
[問卦] 慟！閃亮亮退出硬漢兄弟
https://www.ptt.cc/bbs/Gossiping/M.1586973817.A.EAF.html
Appended!
('list index out of range',)
<div class="r-ent">
<div class="nrec"><span class="hl f2">6</span></div>
<div class="title">
			
				(本文已被刪除) [mlady0923]
			
			</div>
<div class="meta">
<div class="author">-</div>
<div class="article-menu">
</div>
<div class="date"> 4/16</div>
<div class="mark"></div>
</div>
</div>
[問卦] 有睡不著的金城嗎 幫我解一下數學
https://www.ptt.cc/bbs/Gossiping/M.1586974012.A.209.html
Appended!
Re: [問卦] 怎麼判斷鋼琴琴藝高下？
https://www.ptt.cc/bbs/Gossiping/M.1586974074.A.DD2.html
Appended!
[問卦] 體育沒從小開始練無法達到頂尖?
https://www.ptt.cc/bbs/Gossiping/M.1586974094.A.639.html
Appended!
Re: [問卦] 將心比心 如果有天被叫「台灣肺炎」？
https://www.ptt.cc/bbs/Gossipi

Appended!
[問卦] 羅斯柴爾德家族有沒有捐who
https://www.ptt.cc/bbs/Gossiping/M.1586969900.A.A8F.html
Appended!
[問卦] 癩蝦蟆魔咒法
https://www.ptt.cc/bbs/Gossiping/M.1586969940.A.8D0.html
Appended!
Re: [問卦] 日本內閣官員戴布口罩？
https://www.ptt.cc/bbs/Gossiping/M.1586969946.A.8FC.html
Appended!
[問卦] 趁現在夜深人靜我來放點福利圖
https://www.ptt.cc/bbs/Gossiping/M.1586969970.A.F16.html
Appended!
Re: [問卦] 中國真能打下台灣？
https://www.ptt.cc/bbs/Gossiping/M.1586970012.A.41D.html
Appended!
Re: [爆卦] 德國外長：美國不該停止WHO資金援助
https://www.ptt.cc/bbs/Gossiping/M.1586970038.A.4B3.html
Appended!
Re: [新聞] 開600萬瑪莎拉蒂載女友買星巴克！校門前
https://www.ptt.cc/bbs/Gossiping/M.1586970088.A.CD7.html
Appended!
[問卦] 門上的符咒
https://www.ptt.cc/bbs/Gossiping/M.1586970156.A.EC2.html
Appended!
Re: [問卦] 在PTT敢用本人大頭貼的是什麼樣的人
https://www.ptt.cc/bbs/Gossiping/M.1586970164.A.58D.html
Appended!
[問卦] 怎麼看聖凱師的「不是業配」？
https://www.ptt.cc/bbs/Gossiping/M.1586970198.A.E09.html
Appended!
[問卦] 在酒店上班的我看到小妹妹閒逛
https://www.ptt.cc/bbs/Gossiping/M.1586968570.A.A81.html
Appended!
[新聞] 川普才說不給WHO錢　陸外交部馬上「挺

Appended!
[問卦] 買iphone se2的是哪種人？
https://www.ptt.cc/bbs/Gossiping/M.1586967662.A.A5B.html
Appended!
[問卦] 慟!風雲樓二樓漢堡人要島了
https://www.ptt.cc/bbs/Gossiping/M.1586967716.A.8C0.html
Appended!
[問卦] 哀鳳11跟哀鳳SE2 都擠?
https://www.ptt.cc/bbs/Gossiping/M.1586967784.A.52F.html
Appended!
[問卦] 「你阿！滴了？」
https://www.ptt.cc/bbs/Gossiping/M.1586966819.A.6CA.html
Appended!
[新聞] 呂秀蓮示警：別讓「以疫攻台」戰火在台灣
https://www.ptt.cc/bbs/Gossiping/M.1586966841.A.5FA.html
Appended!
[新聞] 藝人祝釩剛 持有大麻電子煙被捕
https://www.ptt.cc/bbs/Gossiping/M.1586966856.A.A88.html
Appended!
[問卦] 大同電鍋有多強？現在出國還會必帶嗎？
https://www.ptt.cc/bbs/Gossiping/M.1586967000.A.790.html
Appended!
Re: [新聞] 誓言「要讓年輕人買不起房！」投資客帥過
https://www.ptt.cc/bbs/Gossiping/M.1586967001.A.B22.html
Appended!
[問卦] 尻尻要衝刺要轉那片？急 在線等
https://www.ptt.cc/bbs/Gossiping/M.1586967004.A.83A.html
Appended!
[問卦] 肥宅會幫正妹加減分嗎？
https://www.ptt.cc/bbs/Gossiping/M.1586967016.A.B7B.html
Appended!
[問卦] 無法跟姊姊相認 是不是很心酸
https://www.ptt.cc/bbs/Gossiping/M.1586967026.A.E43.html
Appended!
[問卦] 如何查閱政府檢測人數
http

In [3]:
len(test_data)

4862

In [175]:
test_data[100:105]

[{'article_url': 'https://www.ptt.cc/bbs/Gossiping/M.1587091888.A.2E8.html',
  'auth_id': 'SakuraYui',
  'auth_name': '櫻花蝦',
  'title': '[新聞] 超車自撞牽拖別人竟連孕婦都打！9屁孩國',
  'date': datetime.datetime(2020, 4, 17, 10, 51, 25),
  'board_name': 'Gossiping',
  'article_content': '\n※新聞內容含有政治人物、政府機關類、政治族群類請以[政治]為分類，違者刪除!\n1.媒體來源:\n※ 例如蘋果日報、自由時報（請參考版規下方的核准媒體名單）\nettoday\n\n2.記者署名\n※ 沒有在這打上記者署名的新聞會被水桶14天 編輯非記者\n※ 外電至少要有來源或編輯 如:法新社\n徐愷昕\n\n3.完整新聞標題:\n※ 標題沒有完整寫出來 ---> 依照板規刪除文章\n超車自撞牽拖別人竟連孕婦都打！9屁孩國道逼車包抄無辜\u3000球棒狠K下場曝\n\n4.完整新聞內文:\n※ 社論特稿都不能貼！違者刪除（政治類水桶3個月)，貼廣告也會被刪除喔！可詳看版規\n\n記者徐愷昕／南投報導\n\n15名年輕人2017年7月相約墾丁旅遊，一行人北返回家途中行經國道3號南投路段時，4部\n車中的其中1部車為超越1輛紅色廂型車，不甚自撞護欄急停，怎料又遭後方來車撞上。超\n車自撞男子的同行友人誤以為是箱型車蛇行致車禍發生，竟集結其他3車在國道上高速逼\n車、倒車、逆向行駛，包抄砸車攻擊廂型車上的駕駛及乘客。南投地院依妨害公眾往來安\n全罪，判處釀禍的9人各3至9月有期徒刑。可上訴。\n\n2017年7月間一群20多歲年輕人開著4部車南下墾丁遊玩，旅程結束北上返家途中，在行經\n國道3號226.8K南投路段時，徐姓男子駕駛的車輛因想超越前方紅色廂型車，怎料過程中\n竟失控擦撞護欄急停在中線車道間，隨後又被後方轎車迎面硬生撞上。\n\n因超車差點受波及的紅色廂型車駕駛見事故發生，上前搖下車窗詢問自撞的徐男有無狀況\n後，見他無礙便隨即離開事故現場。此時整群年輕人的其他3部車，誤以為是紅色廂型車\n蛇行導致事故發生，但卻像沒事一樣直接

# Test zone

In [115]:
article_url = 'https://www.ptt.cc/bbs/Gossiping/M.1119247736.A.35B.html'
getContent(article_url)

{'article_url': 'https://www.ptt.cc/bbs/Gossiping/M.1119247736.A.35B.html',
 'auth_id': 'three3',
 'auth_name': 'Me',
 'title': 'Re: [問題]職棒似乎又出現問題了...',
 'date': datetime.datetime(2005, 6, 20, 14, 20, 54),
 'article_content': '\n我朋友是在新竹執業的理髮師\n\n他有一個固定的顧客就是新竹賭職棒的組頭\n\n那位顧客常常都會放一些消息給我朋友\n\n根據他的消息 錢是每隊都有收\n\n水也是有放 但是程度不一定\n\n每個地區都有不一樣的人上來收\n\n他說  像球衣綠色 母公司是賣農藥的那ㄧ隊\n\n上新竹收的就是很老還不退休的二壘手\n\n球衣綠色和橘色 母公司很大的那ㄧ隊\n\n上新竹收的是弟弟在美國打拚的外野手\n\n總而言之 幾乎都是老選手\n\n有時他會在賽前報比數 大部分都很準 但也是有不準的時候\n\n我們都聽聽就算了 畢竟他們的世界我們很難理解...\n\n\n--\n今晚的比賽究竟誰會獲勝 事實上每一場比賽都一樣\n\n如果今晚你有非贏不可的理由 那你就一定會贏\n\n--\n',
 'comment_content': [{'push-tag': '推 ',
   'push-userid': 'hereafter',
   'push-content': ':是多久以前的事啊?後面那隊從開季到現在只有',
   'push-ipdatetime': '  61.222.252.20 06/20'},
  {'push-tag': '→ ',
   'push-userid': 'hereafter',
   'push-content': ':一場在新竹哩...還要球員跑上來拿',
   'push-ipdatetime': '  61.222.252.20 06/20'},
  {'push-tag': '→ ',
   'push-userid': 'justdoit',
   'push-content': ':有沒有達叔的八卦啊,統一居然想解約他',
   'push-ipdatetime': '140.113

In [1]:
import time


In [21]:
time.time()
time.strftime('%Y/%m/%d', time.localtime())

'2020/04/16'

In [8]:
time.strftime('%m/%d', time.time())

TypeError: Tuple or struct_time argument required

In [10]:
import datetime
datetime.datetime.now()

datetime.datetime(2020, 4, 16, 2, 42, 37, 128850)

In [12]:
datetime.datetime.now() - datetime.timedelta(days=10)

datetime.datetime(2020, 4, 6, 2, 43, 40, 745547)

In [13]:
datetime_now = datetime.datetime.now()
datetime_nowd10 = datetime_now - datetime.timedelta(days=10)
print(datetime_now.strftime('%m/%d'))
print(datetime_nowd10.strftime('%m/%d'))

04/16
04/06


In [15]:
datetime_now < datetime_nowd10

False

In [16]:
datetime.datetime(1970, 1, 1)

datetime.datetime(1970, 1, 1, 0, 0)

In [20]:
datetime.datetime.strptime('0406', '%m%d')

datetime.datetime(1900, 4, 6, 0, 0)

In [39]:
import requests
from bs4 import BeautifulSoup
import random
import time
import datetime
import logging
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36'}
cookies = {'over18': '1'}
board_url = 'https://www.ptt.cc/bbs/Gossiping/index.html'
datetime_now = datetime.datetime.now()
datetime_1970 = datetime.datetime(1970, 1, 1)
end_date = datetime_now.strftime('%m/%d')
start_date = datetime_1970.strftime('%m/%d')
ss = requests.session()
ss.cookies['over18'] = '1'
res = ss.get(board_url, headers=headers)
soup = BeautifulSoup(res.text, 'html.parser')
titles_soup_list = soup.select('div.r-ent')
# last_page_url = 'https://www.ptt.cc' + \
#                     soup.select('div[class="btn-group btn-group-paging"] a.btn')[1]['href']
for title_soup in titles_soup_list:
    article_url = 'https://www.ptt.cc' + title_soup.select('div.title a')[0]['href']
    article_title = title_soup.select('div.title a')[0].text
    print(article_title)
    print(article_url)
    print()


[問卦] 幾歲退休算溫拿
https://www.ptt.cc/bbs/Gossiping/M.1587022663.A.95E.html

[問卦] 養吉娃娃信教吃素四十歲國文老師可怕嗎
https://www.ptt.cc/bbs/Gossiping/M.1587022676.A.0CE.html

Re: [新聞] 越南維持「零死亡」神話 德媒：防疫比台灣亮眼
https://www.ptt.cc/bbs/Gossiping/M.1587022773.A.BF7.html

[問卦] 黑人到底有什麼特長？
https://www.ptt.cc/bbs/Gossiping/M.1587022796.A.88C.html

Re: [爆卦] 武漢人托我傳達出來的訊息！
https://www.ptt.cc/bbs/Gossiping/M.1587022829.A.485.html

[問卦] 計程車司機沒收入能幹嘛??
https://www.ptt.cc/bbs/Gossiping/M.1587022830.A.663.html

發了1400篇文章 你各位可以叫我一聲學姊了吧
https://www.ptt.cc/bbs/Gossiping/M.1587022859.A.061.html

[新聞] 影／「台灣」卡在喉嚨說不出！德100萬片口罩到手竟馬上取消
https://www.ptt.cc/bbs/Gossiping/M.1587022869.A.C24.html

[公告] 八卦板板規(2020.03.21)
https://www.ptt.cc/bbs/Gossiping/M.1584722568.A.473.html

[協尋]代PO-4/12基隆路與羅斯福路口車禍影像
https://www.ptt.cc/bbs/Gossiping/M.1586864827.A.F44.html

[公告] 數條板規修改草案於gossippicket討論
https://www.ptt.cc/bbs/Gossiping/M.1586874858.A.733.html

[公告] ㊣◎☆★八卦版主徵選報名開始★☆◎㊣
https://www.ptt.cc/bbs/Gossiping/M.1586904028.A.BCF.html

[公告] 四月份置底閒聊文
ht

In [30]:
last_page_url

'https://www.ptt.cc/bbs/Gossiping/index2.html'

In [112]:
import requests
from bs4 import BeautifulSoup
import random
import time
import datetime

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36'}
cookies = {'over18': '1'}
board_url = 'https://www.ptt.cc/bbs/Gossiping/index.html'
datetime_now = datetime.datetime.now()
datetime_1970 = datetime.datetime(1970, 1, 1)
end_date = datetime_now.strftime('%m/%d')
start_date = datetime_1970.strftime('%m/%d')

ss = requests.session()
ss.cookies['over18'] = '1'

article_url = 'https://www.ptt.cc/bbs/Gossiping/M.1119247736.A.35B.html'

article_content = dict()
article_content['article_url'] = article_url
res = ss.get(article_url, headers=headers)
soup = BeautifulSoup(res.text, 'html.parser')

# Get author information : 作者 -> 標題 -> 時間
# Dict[article_url, auth_id, auth_name, date, article_content, comment_content]
article_content['auth_id'] = soup.select('div[class="article-metaline"]')[0] \
                                .span.find_next_sibling() \
                                .text \
                                .split(' ')[0]
article_content['auth_name'] = soup.select('div[class="article-metaline"]')[0] \
                                .extract() \
                                .span.find_next_sibling() \
                                .text \
                                .split(' ')[1][1:-1]
article_content['title'] = soup.select('div[class="article-metaline"]')[0] \
                                .extract() \
                                .span.find_next_sibling() \
                                .text 
article_content['date'] = soup.select('div[class="article-metaline"]')[0] \
                                .extract() \
                                .span.find_next_sibling() \
                                .text
article_content['date'] = datetime.datetime.strptime(' '.join(article_content['date'].split(' ')[1:])
                                                     , '%b %d %X %Y')

soup.select('div[class="article-metaline-right"]')[0].extract()
    
# Get article content
article_content['article_content'] = soup.select('#main-content')[0].text.split('※ 發信站')[0]

# Get the whole comments without classifying by comment ID
article_content['comment_content'] = list()
tmp_dict = dict()
for tag in soup.select('#main-content')[0].select('span')[2:]:
    for comment_key in ['push-tag', 'push-userid', 'push-content', 'push-ipdatetime']:
        if comment_key in tag.attrs['class']:
            tmp_dict[comment_key] = tag.text.strip('\n')
            # The order of class in span-tag is ['push-tag', 'push-userid', 'push-content', 'push-ipdatetime']
            if comment_key == 'push-ipdatetime': # Reset tmp_dict()
                article_content['comment_content'].append(tmp_dict)
                tmp_dict = dict()
                pass
                

In [113]:
article_content['comment_content']

[{'push-tag': '推 ',
  'push-userid': 'hereafter',
  'push-content': ':是多久以前的事啊?後面那隊從開季到現在只有',
  'push-ipdatetime': '  61.222.252.20 06/20'},
 {'push-tag': '→ ',
  'push-userid': 'hereafter',
  'push-content': ':一場在新竹哩...還要球員跑上來拿',
  'push-ipdatetime': '  61.222.252.20 06/20'},
 {'push-tag': '→ ',
  'push-userid': 'justdoit',
  'push-content': ':有沒有達叔的八卦啊,統一居然想解約他',
  'push-ipdatetime': '140.113.144.250 06/20'},
 {'push-tag': '推 ',
  'push-userid': 'etherent',
  'push-content': ':大概是教練收了吧.....',
  'push-ipdatetime': '218.163.164.134 06/20'},
 {'push-tag': '推 ',
  'push-userid': 'comduffer',
  'push-content': ':我也有買過球員...........................卡',
  'push-ipdatetime': ' 03/31 11:48'},
 {'push-tag': '推 ',
  'push-userid': 'lionsss',
  'push-content': ':聽你在放臭屁...',
  'push-ipdatetime': ' 06/10 20:34'},
 {'push-tag': '推 ',
  'push-userid': 'ii9',
  'push-content': ':所以 可以到雙溪嗎？！',
  'push-ipdatetime': ' 08/31 20:59'},
 {'push-tag': '推 ',
  'push-userid': 'IEhacker',
  'push-content': ':專業',

In [109]:
'push-tag' in soup.select('#main-content')[0].select('span')[2:][5].attrs['class']

True

In [91]:
soup.select('#main-content')[0].text.split('--')[1]

'\n※ 發信站: 批踢踢實業坊(ptt.cc), 來自: 220.143.27.86 (臺灣)\n※ 文章網址: https://www.ptt.cc/bbs/Gossiping/M.1587015942.A.1C2.html\n噓 zxc2331189: 釣的到屁眼榨甘蔗 04/16 13:46\n→ kino056: 都是大雄的夢 04/16 13:46\n噓 AOA2: 你才知道時空刑警局幫他們擦了多少屁股~哆啦A夢在22世紀搞不 04/16 13:53\n→ AOA2: 好已經是十大通緝犯之首 04/16 13:53\n→ goto: 在下面看得到他頭頂？ 04/16 13:58\n'

In [81]:
for i in article_content:
    print(type(article_content[i]))

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>


In [85]:
article_content

{'article_url': 'https://www.ptt.cc/bbs/Gossiping/M.1587015942.A.1C2.html',
 'auth_id': 'a6234709',
 'auth_name': '矮醜魯肥宅之代言人',
 'title': '[問卦] 有沒有哆啦A夢的道具上新聞的卦??',
 'date': datetime.datetime(2020, 4, 16, 13, 45, 40)}

In [78]:
soup.select('div[class="article-metaline"]')[0] \
.span

<span class="article-meta-tag">作者</span>

In [55]:
soup.select('div[class="article-metaline"]')

[]

In [69]:
for auth_info in soup.select('div[class="article-metaline"]'):
    auth_info.select('span.article-meta-value')

<div class="article-metaline"><span class="article-meta-tag">作者</span><span class="article-meta-value">a6234709 (矮醜魯肥宅之代言人)</span></div>
<div class="article-metaline"><span class="article-meta-tag">標題</span><span class="article-meta-value">[問卦] 有沒有哆啦A夢的道具上新聞的卦??</span></div>
<div class="article-metaline"><span class="article-meta-tag">時間</span><span class="article-meta-value">Thu Apr 16 13:45:40 2020</span></div>


In [170]:
'https://www.ptt.cc/bbs/Gossiping/M.1587086707.A.07A.html'.split('/')[-2]

'Gossiping'

In [176]:
import json

s = json.dumps(test_data[100:105])
s

TypeError: Object of type datetime is not JSON serializable

In [178]:
aaa = eval(str(test_data[100:105]))

In [181]:
type(aaa[0]['date'])

datetime.datetime

In [182]:
str(test_data[100:105])

'[{\'article_url\': \'https://www.ptt.cc/bbs/Gossiping/M.1587091888.A.2E8.html\', \'auth_id\': \'SakuraYui\', \'auth_name\': \'櫻花蝦\', \'title\': \'[新聞] 超車自撞牽拖別人竟連孕婦都打！9屁孩國\', \'date\': datetime.datetime(2020, 4, 17, 10, 51, 25), \'board_name\': \'Gossiping\', \'article_content\': \'\\n※新聞內容含有政治人物、政府機關類、政治族群類請以[政治]為分類，違者刪除!\\n1.媒體來源:\\n※ 例如蘋果日報、自由時報（請參考版規下方的核准媒體名單）\\nettoday\\n\\n2.記者署名\\n※ 沒有在這打上記者署名的新聞會被水桶14天 編輯非記者\\n※ 外電至少要有來源或編輯 如:法新社\\n徐愷昕\\n\\n3.完整新聞標題:\\n※ 標題沒有完整寫出來 ---> 依照板規刪除文章\\n超車自撞牽拖別人竟連孕婦都打！9屁孩國道逼車包抄無辜\\u3000球棒狠K下場曝\\n\\n4.完整新聞內文:\\n※ 社論特稿都不能貼！違者刪除（政治類水桶3個月)，貼廣告也會被刪除喔！可詳看版規\\n\\n記者徐愷昕／南投報導\\n\\n15名年輕人2017年7月相約墾丁旅遊，一行人北返回家途中行經國道3號南投路段時，4部\\n車中的其中1部車為超越1輛紅色廂型車，不甚自撞護欄急停，怎料又遭後方來車撞上。超\\n車自撞男子的同行友人誤以為是箱型車蛇行致車禍發生，竟集結其他3車在國道上高速逼\\n車、倒車、逆向行駛，包抄砸車攻擊廂型車上的駕駛及乘客。南投地院依妨害公眾往來安\\n全罪，判處釀禍的9人各3至9月有期徒刑。可上訴。\\n\\n2017年7月間一群20多歲年輕人開著4部車南下墾丁遊玩，旅程結束北上返家途中，在行經\\n國道3號226.8K南投路段時，徐姓男子駕駛的車輛因想超越前方紅色廂型車，怎料過程中\\n竟失控擦撞護欄急停在中線車道間，隨後又被後方轎車迎面硬生撞上。\\n\\n因超車差點受波及的紅色廂型車駕駛見事故發生，上前搖下車窗詢問自撞的徐男有無狀況\\n後，見他無礙便隨即離開事故

In [12]:
import datetime
datetime.time()

datetime.time(0, 0)

In [14]:
import time
t = time.time()
dt = datetime.datetime.fromtimestamp(t)
dt

datetime.datetime(2020, 4, 21, 3, 57, 19, 487582)

In [15]:
import pytz
dt.astimezone(pytz.utc)

datetime.datetime(2020, 4, 21, 3, 57, 19, 487582, tzinfo=<UTC>)

In [22]:
datetime.datetime.now().astimezone(pytz.utc)

datetime.datetime(2020, 4, 21, 4, 2, 56, 96909, tzinfo=<UTC>)

In [18]:
datetime.datetime.now()

datetime.datetime(2020, 4, 21, 4, 1, 40, 84298)

In [24]:
t = time.time()
dt = datetime.datetime.fromtimestamp(t)
print(t)
dt

1587441898.1999834


datetime.datetime(2020, 4, 21, 4, 4, 58, 199983)

In [2]:
getContent(article_url='https://www.ptt.cc/bbs/Gossiping/M.1587369966.A.0B8.html', process_id='test20200422')

KAFKA PRODUCED
Successfully connected!
KAFKA PRODUCED
Successfully connected!


{'article_url': 'https://www.ptt.cc/bbs/Gossiping/M.1587369966.A.0B8.html',
 'auth_id': 'LeonardoChen',
 'auth_name': 'LeonardoChen',
 'title': '[問卦] 賣車的小姐怎麼這麼嗆？',
 'date': datetime.datetime(2020, 4, 20, 16, 6, 1),
 'board_name': 'Gossiping',
 'article_content': '\n強者我朋友阿砲想脫離騎車的賤民階級，我陪他去看車\n\n賣車的小姐看起來溫溫柔柔，我說我沒有要買\n\n她居然說，你不買你可以試試看\n\n賣車的小姐怎麼這麼嗆？\n\n不買試試看？\n\n--\n推 dk****:這讓我想起之前打工時認識一個念文化大二的正妹  她跟我說10/17 01:28\n→ AceCh****:臭宅宅10/17 01:28\n→ dk****:她們學校的男生都很愛玩  她想找個乖一點的  我覺得她對我10/17 01:30\n→ AceCh****:露出第三條腿10/17 01:30\n→ dk****:好像有意思  可是沒多久她說她交了男朋友  是念文化的10/17 01:32\n推 GoodElep****:掃地阿伯10/17 01:33\n\n--\n',
 'comment_content': [{'push-content': ':這讓我想起之前打工時認識一個念文化大二的正妹  她跟我說',
   'push-ipdatetime': '10/17 01:28'},
  {'push-tag': '→ ',
   'push-userid': 'AceCh****',
   'push-content': ':臭宅宅',
   'push-ipdatetime': '10/17 01:28'},
  {'push-tag': '→ ',
   'push-userid': 'dk****',
   'push-content': ':她們學校的男生都很愛玩  她想找個乖一點的  我覺得她對我',
   'push-ipdatetime': '10/17 01:30'},
  {'push-tag': '→ '